# Export model

In [1]:
import os
from pathlib import Path

current_dir = str(Path.cwd())
current_dir

'/home/ec2-user/SageMaker/amazon-sagemaker-tensorflow-form-field-det/2_train_model'

In [2]:
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

container

'753177091730.dkr.ecr.us-east-1.amazonaws.com/sagemaker-form-field-detection:20210809131501'

In [35]:
short_name='exporter'
!docker ps -a

CONTAINER ID        IMAGE                                                                                        COMMAND             CREATED             STATUS              PORTS               NAMES
6afe84d8f8a3        753177091730.dkr.ecr.us-east-1.amazonaws.com/sagemaker-form-field-detection:20210809131501   "/bin/bash"         16 hours ago        Up 16 hours                             exporter


In [36]:
!docker stop $short_name

exporter


In [37]:
!docker rm $short_name

Error: No such container: exporter


In [38]:
#!docker run --name cml_runner_az --gpus all -d -v /mnt/data01/processing-stages:/mydvc/processing-stages  -v /root/rosecyt-trainer/dvc_cache/:/mydvc/dvc_cache -e RUNNER_LABELS=cml,gpu,baremetal
docker_mapped_source_dir = '/home/source_dir'

!docker run --name $short_name -d --rm -i -t -w $docker_mapped_source_dir -v $current_dir/source_dir:$docker_mapped_source_dir -e MODEL_DIR=$docker_mapped_source_dir/checkpoint $container

# %%bash
# docker run --name $short_name -d \
#     --rm -i -t \
#     -v /home/ec2-user/SageMaker/amazon-sagemaker-tensorflow-form-field-det/2_train_model/source_dir:$docker_mapped_source_dir \
#     -e MODEL_DIR=$docker_mapped_source_dir \
#     $container

9dc70eb49ec252d705cbac82c3baa54d1be75179ddd24c55460a1a6f1837e1b8


In [39]:
#check work
!docker ps
!docker exec -it $short_name echo "worked"

CONTAINER ID        IMAGE                                                                                        COMMAND             CREATED             STATUS              PORTS               NAMES
9dc70eb49ec2        753177091730.dkr.ecr.us-east-1.amazonaws.com/sagemaker-form-field-detection:20210809131501   "/bin/bash"         4 seconds ago       Up 3 seconds                            exporter
worked


In [40]:
#!docker container inspect $short_name

In [41]:
!docker exec -it $short_name pwd

/home/source_dir


In [26]:
!docker exec -it $short_name ls checkpoint

checkpoint  ckpt-0.data-00000-of-00001	ckpt-0.index


In [42]:
version=1
pipeline_config = str(Path(docker_mapped_source_dir) / 'ssd_efficientdet_d4_1024x1024_formdet.config') 
checkpoint_dir = str(Path(docker_mapped_source_dir) / 'checkpoint')
saved_model = str(Path(docker_mapped_source_dir) / f'model_{version}')
print(f'pipeline config: {pipeline_config} saved_model: {saved_model} checkpoint: {checkpoint_dir}')

pipeline config: /home/source_dir/ssd_efficientdet_d4_1024x1024_formdet.config saved_model: /home/source_dir/model_1 checkpoint: /home/source_dir/checkpoint


In [44]:
 !sudo rm -rf source_dir/saved_model

In [45]:
!docker exec -it $short_name python exporter_main_v2.py --trained_checkpoint_dir $checkpoint_dir/ --pipeline_config_path $pipeline_config --output_directory $saved_model/
# !docker exec -it $short_name python exporter_main_v2.py --trained_checkpoint_dir ./checkpoint --pipeline_config_path ./ssd_efficientdet_d4_1024x1024_formdet.config --output_directory ./saved_model


Trained checkpoint dir: /home/source_dir/checkpoint/ exists: True
I0812 19:55:47.625761 140139250874176 ssd_efficientnet_bifpn_feature_extractor.py:143] EfficientDet EfficientNet backbone version: efficientnet-b4
I0812 19:55:47.625987 140139250874176 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet BiFPN num filters: 224
I0812 19:55:47.626080 140139250874176 ssd_efficientnet_bifpn_feature_extractor.py:146] EfficientDet BiFPN num iterations: 7
I0812 19:55:47.632981 140139250874176 efficientnet_model.py:147] round_filter input=32 output=48
I0812 19:55:47.664329 140139250874176 efficientnet_model.py:147] round_filter input=32 output=48
I0812 19:55:47.664499 140139250874176 efficientnet_model.py:147] round_filter input=16 output=24
I0812 19:55:47.802603 140139250874176 efficientnet_model.py:147] round_filter input=16 output=24
I0812 19:55:47.802788 140139250874176 efficientnet_model.py:147] round_filter input=24 output=32
I0812 19:55:48.143701 140139250874176 efficientnet_mode

In [46]:
print('done')

done


### Rename saved_model dir to version num 

In [50]:
!sudo mv source_dir/model_$version/saved_model source_dir/model_$version/$version

## Create a model archive file

SageMaker models need to be packaged in .tar.gz files. When your endpoint is provisioned, the files in the archive will be extracted and put in /opt/ml/model/ on the endpoint.

In [60]:

# !saved_model_cli show --all --dir /home/ec2-user/SageMaker/amazon-sagemaker-tensorflow-form-field-det/2_train_model/source_dir/saved_model/saved_model

!tar -C source_dir/model_$version -cvzf form-field-det.tar.gz $version

1/
1/variables/
1/variables/variables.data-00000-of-00001
1/variables/variables.index
1/assets/
1/saved_model.pb


In [61]:
!pwd


/home/ec2-user/SageMaker/amazon-sagemaker-tensorflow-form-field-det/2_train_model
